In [1]:
import os

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [2]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.visualization import raster_plot

C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get sessions data

Sessions are already loaded on local drive.

In [3]:
# cache directory path, it determines where downloaded data will be stored
output_dir = '/ecephys_cache_dir/'
manifest_path = os.path.join(output_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
print(cache.get_all_session_types())

# functional connecivity dataset contains 30 min spontaneous activity block
sessions = cache.get_session_table()
print('len sessions: {}'.format(len(sessions)))

brain_observatory_type_sessions = sessions[sessions["session_type"] == "functional_connectivity"]
print('len brain_observatory_type_sessions = functional_connectivity: {}'.format(len(brain_observatory_type_sessions)))
print(brain_observatory_type_sessions.keys())

brain_observatory_type_sessions.tail(3)

['brain_observatory_1.1', 'functional_connectivity']
len sessions: 58
len brain_observatory_type_sessions = functional_connectivity: 26
Index(['published_at', 'specimen_id', 'session_type', 'age_in_days', 'sex',
       'full_genotype', 'unit_count', 'channel_count', 'probe_count',
       'ecephys_structure_acronyms'],
      dtype='object')


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
839557629,2019-10-03T00:00:00Z,821469666,functional_connectivity,115.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,450,1853,5,"[APN, NOT, MB, DG, CA1, VISam, nan, VISpm, LGd..."
840012044,2019-10-03T00:00:00Z,820866121,functional_connectivity,116.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,758,2298,6,"[APN, DG, CA1, VISam, nan, LP, VISpm, VISp, LG..."
847657808,2019-10-03T00:00:00Z,827809884,functional_connectivity,126.0,F,wt/wt,874,2298,6,"[APN, NOT, DG, HPF, ProS, CA1, VISam, nan, MB,..."


### Get single units 

using 30 min of spontaneous activity (animals were shown grey screen)

**todo:**  figure out quality cutoffs;
for explanation why low amplitude cutoff is good for ITs 
see also https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html#Amplitude-cutoff
the idea is to get units that are more "complete", 0.01 means 1% of spikes is missing from the units

In [4]:
output_filename_ = 'Q:\\Personal\\Irina\\projects\\isttc\\' + 'allen_test_full_v2.csv'
verbose = False

In [5]:
for session_id in brain_observatory_type_sessions.index.values:

    print('############################')
    print('processing session {}'.format(session_id))
    
    # load session
    session = cache.get_session_data(session_id)
    
    # load units
    units_df = session.units
    print('len units df {}'.format(len(units_df)))
    
    # load stimulus presentation 
    presentations = session.get_stimulus_table("spontaneous")
    spont_period_id = presentations.query('duration > 1200').index.values[0]
    print('Spontaneous period {}'.format(presentations.loc[spont_period_id, :]))
    
    # load spikes from stimulus period
    spikes_df = session.presentationwise_spike_times(
        stimulus_presentation_ids=spont_period_id,  
        unit_ids=units_df.index.values
    )
    
    # make df
    spikes_df['time_since_stimulus_presentation_onset_str'] = spikes_df.time_since_stimulus_presentation_onset.astype(str)
    spikes_wide_df = spikes_df.groupby(by='unit_id', as_index=False).agg(spike_times=('time_since_stimulus_presentation_onset_str', ','.join))
    
    units_df_subset = units_df[['ecephys_structure_acronym']].copy()
    units_df_subset.reset_index(inplace=True)
    units_df_subset['specimen_id'] = brain_observatory_type_sessions.loc[session_id, :]['specimen_id']
    units_df_subset['session_id'] = session_id
    
    units_merged_df = pd.merge(units_df_subset, spikes_wide_df, on='unit_id', how='inner')
    
    # write to file 
    spikes_out_dict = units_merged_df.to_dict(orient='index')
    print('Writing to csv...')
    with open(output_filename_, 'a', newline='') as f:
        writer = csv.writer(f)
        for k,v in spikes_out_dict.items():
            if verbose:
                print('Writing unit {}'.format(v['unit_id']))
            #    spikes_l = spike_train.tolist()
            row = [v['specimen_id']] + [v['session_id']] + [v['unit_id']] + [v['ecephys_structure_acronym']] + list(map(float, v['spike_times'].split(',')))
            writer.writerow(row)

############################
processing session 766640955


C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 be

len units df 842


C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
C:\Users\ipochino\.conda\envs\allensdk2\Lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)


Spontaneous period start_time               4397.968317
stop_time                6200.474424
stimulus_name            spontaneous
duration                 1802.506107
stimulus_condition_id              0
Name: 40639, dtype: object
Writing to csv...
############################
processing session 767871931
len units df 713
Spontaneous period start_time               4397.997358
stop_time                6200.504082
stimulus_name            spontaneous
duration                 1802.506725
stimulus_condition_id              0
Name: 40639, dtype: object
Writing to csv...
############################
processing session 768515987
len units df 802
Spontaneous period start_time                4397.96871
stop_time                6200.474769
stimulus_name            spontaneous
duration                 1802.506059
stimulus_condition_id              0
Name: 40639, dtype: object
Writing to csv...
############################
processing session 771160300
len units df 930
Spontaneous period start_tim

In [ ]:
# spikes_out_dict = units_merged_df.to_dict(orient='index')
# spikes_out_dict[0]